In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1686276429009_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
park=SparkSession.builder.appName('data_processing').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df=spark.read.csv('s3://st0263darangoh/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'Nombre departamento', 'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicación del caso', 'Estado', 'Código ISO del país', 'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico']

In [5]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [6]:
df.select('Fecha de muerte','Nombre departamento','Sexo').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------+----+
|Fecha de muerte|Nombre departamento|Sexo|
+---------------+-------------------+----+
|           null|             BOGOTA|   F|
|           null|              VALLE|   M|
|           null|          ANTIOQUIA|   F|
|           null|          ANTIOQUIA|   M|
|           null|          ANTIOQUIA|   M|
|           null|          ANTIOQUIA|   F|
|           null|          CARTAGENA|   F|
|           null|             BOGOTA|   F|
|           null|             BOGOTA|   F|
|           null|             BOGOTA|   F|
+---------------+-------------------+----+
only showing top 10 rows

In [7]:
df =df.withColumnRenamed("fecha reporte web", "fecha_reporte_web")
df =df.withColumnRenamed("ID de caso", "id_caso")
df =df.withColumnRenamed("Fecha de notificación", "fecha_notificacion")
df =df.withColumnRenamed("Código DIVIPOLA departamento", "codigo_departamento")
df =df.withColumnRenamed("Nombre departamento", "departamento")
df =df.withColumnRenamed("Código DIVIPOLA municipio", "codigo_municipio")
df =df.withColumnRenamed("Nombre municipio", "municipio")
df =df.withColumnRenamed("Edad", "edad")
df =df.withColumnRenamed("Sexo", "sexo")
df =df.withColumnRenamed("Tipo de contagio", "tipo_contagio")
df =df.withColumnRenamed("Ubicación del caso", "ubicacion_caso")
df =df.withColumnRenamed("Estado", "estado")
df =df.withColumnRenamed("Código ISO del país", "codigo_pais")
df =df.withColumnRenamed("Nombre del país", "nombre_pais")
df =df.withColumnRenamed("Recuperado", "recuperado")
df =df.withColumnRenamed("Fecha de inicio de síntomas", "fecha_inicio_sintomas")
df =df.withColumnRenamed("Fecha de muerte", "fecha_muerte")
df =df.withColumnRenamed("Fecha de diagnóstico", "fecha_diagnostico")
df =df.withColumnRenamed("Fecha de recuperación", "fecha_recuperacion")
df =df.withColumnRenamed("Tipo de recuperación", "tipo_recuperacion")
df =df.withColumnRenamed("Pertenencia étnica", "pertenencia_etnica")
df =df.withColumnRenamed("Nombre del grupo étnico", "nombre_etnia")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df = df.withColumn("estado_por_sexo", concat(col("id_caso"), lit(", "), col("estado"), lit(", "), col("sexo")))
df.select("estado_por_sexo").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|estado_por_sexo|
+---------------+
|     1, Leve, F|
|     2, Leve, M|
|     3, Leve, F|
|     4, Leve, M|
|     5, Leve, M|
+---------------+
only showing top 5 rows

In [9]:
df = df.drop("fecha_muerte","codigo_municipio")
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha_reporte_web', 'id_caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'departamento', 'Código DIVIPOLA municipio', 'municipio', 'edad', 'Unidad de medida de edad', 'sexo', 'tipo_contagio', 'Ubicación del caso', 'estado', 'Código ISO del país', 'Nombre del país', 'recuperado', 'Fecha de inicio de síntomas', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico', 'estado_por_sexo']

In [10]:
df.select("id_caso", "estado","sexo").filter(df["sexo"] == "F").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+----+
|id_caso|estado|sexo|
+-------+------+----+
|      1|  Leve|   F|
|      3|  Leve|   F|
|      6|  Leve|   F|
|      7|  Leve|   F|
|      8|  Leve|   F|
|      9|  Leve|   F|
|     10|  Leve|   F|
|     11|  Leve|   F|
|     12|  Leve|   F|
|     13|  Leve|   F|
+-------+------+----+
only showing top 10 rows

In [11]:
lambda_function = udf(lambda column_value: "estado: " + column_value)
df = df.withColumn("tag", lambda_function(df["estado"]))
df.select('estado','tag').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------+
|estado|         tag|
+------+------------+
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
+------+------------+
only showing top 5 rows

In [12]:
departamentos = df.groupBy("departamento").count().orderBy(desc("count")).limit(10)
departamentos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|departamento|count|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+

In [13]:
df.createOrReplaceTempView("data")
departamentos = spark.sql("SELECT departamento, COUNT(*) AS casos FROM data GROUP BY departamento ORDER BY casos DESC LIMIT 10")
departamentos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|departamento|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+

In [14]:
df.createOrReplaceTempView("data")
departamentos = spark.sql("SELECT departamento, COUNT(*) AS casos FROM data GROUP BY departamento ORDER BY casos DESC LIMIT 10")
departamentos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|departamento|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+

In [15]:
municipios = spark.sql("SELECT municipio, COUNT(*) AS casos FROM data GROUP BY municipio ORDER BY casos DESC LIMIT 10")
municipios.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|   municipio|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   CARTAGENA| 8333|
|        CALI| 7747|
|     SOLEDAD| 6233|
|     LETICIA| 2194|
|    MEDELLIN| 2137|
|      TUMACO| 1501|
|BUENAVENTURA| 1453|
|      QUIBDO| 1367|
+------------+-----+

In [16]:
dias = df.groupBy("fecha_reporte_web").count().orderBy(desc("count")).limit(10)
dias.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|fecha_reporte_web|count|
+-----------------+-----+
|27/6/2020 0:00:00| 4149|
|26/6/2020 0:00:00| 3843|
|24/6/2020 0:00:00| 3541|
|25/6/2020 0:00:00| 3486|
|29/6/2020 0:00:00| 3274|
|28/6/2020 0:00:00| 3178|
|18/6/2020 0:00:00| 3171|
|19/6/2020 0:00:00| 3059|
|21/6/2020 0:00:00| 3019|
|30/6/2020 0:00:00| 2803|
+-----------------+-----+

In [17]:
dias = spark.sql("SELECT fecha_reporte_web as Fecha, COUNT(*) AS Casos FROM data GROUP BY fecha_reporte_web ORDER BY casos DESC LIMIT 10")
dias.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|            Fecha|Casos|
+-----------------+-----+
|27/6/2020 0:00:00| 4149|
|26/6/2020 0:00:00| 3843|
|24/6/2020 0:00:00| 3541|
|25/6/2020 0:00:00| 3486|
|29/6/2020 0:00:00| 3274|
|28/6/2020 0:00:00| 3178|
|18/6/2020 0:00:00| 3171|
|19/6/2020 0:00:00| 3059|
|21/6/2020 0:00:00| 3019|
|30/6/2020 0:00:00| 2803|
+-----------------+-----+

In [18]:
edad = df.groupBy("edad").count().orderBy(desc("count")).limit(10)
edad.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|edad|count|
+----+-----+
|  30| 2735|
|  29| 2611|
|  31| 2569|
|  28| 2540|
|  27| 2494|
|  26| 2436|
|  33| 2371|
|  32| 2362|
|  25| 2335|
|  34| 2310|
+----+-----+

In [19]:
edad = spark.sql("SELECT edad, COUNT(*) AS casos FROM data GROUP BY edad ORDER BY casos DESC LIMIT 10")
edad.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|edad|casos|
+----+-----+
|  30| 2735|
|  29| 2611|
|  31| 2569|
|  28| 2540|
|  27| 2494|
|  26| 2436|
|  33| 2371|
|  32| 2362|
|  25| 2335|
|  34| 2310|
+----+-----+

In [20]:
estado = df.groupBy("estado").count().orderBy(desc("count")).limit(10)
estado.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|   estado|count|
+---------+-----+
|     Leve|94367|
|Fallecido| 4663|
|      N/A|  970|
+---------+-----+

In [22]:
estado = spark.sql("SELECT estado, COUNT(*) AS casos FROM data GROUP BY estado ORDER BY casos DESC LIMIT 10")
estado.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|   estado|casos|
+---------+-----+
|     Leve|94367|
|Fallecido| 4663|
|      N/A|  970|
+---------+-----+

In [23]:
departamentos.write.csv("s3://st0263darangoh/lab62/departamentos.csv")
municipios.write.csv("s3://st0263darangoh/lab62/municipios.csv")
dias.write.csv("s3://st0263darangoh/lab62/dias.csv")
edad.write.csv("s3://st0263darangoh/lab62/edad.csv")
estado.write.csv("s3://st0263darangoh/lab62/estado.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…